In [1]:
from src.data_structure import *

IndentationError: unindent does not match any outer indentation level (data_structure.py, line 427)

In [2]:
from src.tag import *

In [3]:
import pprint

In [4]:
text = open("sample.txt").read()

start_tag_added_text = mark_sections(text)

start_tag_and_placeholder_text, text_placeholders = replace_text_with_placeholder(
        start_tag_added_text
    )

prioritized_tag_text = replace_tags_with_rules(start_tag_and_placeholder_text)
tag_complited_text = parse_and_correct_html(prioritized_tag_text)
result = restore_text_from_placeholder(tag_complited_text, text_placeholders)

#pprint.pprint(result)


In [5]:
html_text = result

In [6]:
from bs4 import BeautifulSoup
import re

class Section:
    def __init__(self, title, body):
        self.title = title
        self.body = body

def find_section_boundaries(html, tags):
    tags_regex = '|'.join(tags)
    pattern = re.compile(f"<(/)?({tags_regex})(:\\d*)?>")
    boundaries = []
    for match in pattern.finditer(html):
        tag_full, is_closing, tag_name, span = match.group(0), bool(match.group(1)), match.group(2), match.span()
        boundaries.append((is_closing, tag_name, span))
    return boundaries

def construct_sections(html, boundaries):
    sections = []
    open_tag = None
    section_start = 0
    for is_closing, tag_name, span in boundaries:
        if not is_closing:
            if open_tag is not None:
                sections.append((open_tag, html[section_start:span[0]].strip()))
            open_tag = tag_name
            section_start = span[1]
        else:
            if open_tag == tag_name:
                sections.append((tag_name, html[section_start:span[0]].strip()))
                open_tag = None
    return sections

def extract_content_from_section(section):
    soup = BeautifulSoup(section, 'html.parser')
    title = extract_title_or_body(soup, 'TITLE', True)
    body = extract_title_or_body(soup, 'body')
    
    # Adjust title or body if they are not found
    if not title and not body:
        # If both title and body are missing, use raw text as the title.
        title = soup.get_text(strip=True)
    elif title and not body:
        # If title exists but body is missing, use raw text as the body.
        body = [soup.get_text(strip=True)]
    
    return title, body

def extract_title_or_body(soup, tag, is_title=False):
    if is_title:
        title_tag = soup.find(lambda tag: tag.name.upper().endswith('TITLE'))
        return title_tag.get_text(strip=True) if title_tag else ""
    else:
        body_tag = soup.find(tag)
        return body_tag.get_text(strip=True).split("\n") if body_tag else []

def process_html_into_sections(html_text, tag_list):
    boundaries = find_section_boundaries(html_text, tag_list)
    raw_sections = construct_sections(html_text, boundaries)
    processed_sections = [Section(*extract_content_from_section(section)) for _, section in raw_sections]
    return processed_sections

# 使用例
tag_list = ['PROLOGUE', 'CHAPTER', 'EPILOGUE', 'THE_END', 'TRANSCRIBER_NOTES']

processed_sections = process_html_into_sections(html_text, tag_list)
for section in processed_sections:
    print(f"Title: {section.title}")
    print("Body:", section.body)
    print("---")


Title: PROLOGUE
Body: ['Cigars had burned low, and we were beginning to sample the disillusionment...']
---
Title: CHAPTER 1
Body: ['During that third week of May the situation in Baskul had become much worse...']
---
Title: CHAPTER 2
Body: ['The journey was expected to be long and perilous...', '', 'gave him a most charming smile, but her eyes were all for the boy.', '“What life?”', '', '“The life you’re thinking of . . . dinners . . . dances . . . polo . . .', 'all that. . . .”', '', '“But I never said anything about dances and polo! Anyhow, what’s wrong', 'with them? D’you mean that you’re not coming with me? You’re going to', 'stay here like the other two? Then at least you shan’t stop _me_ from', 'clearing out of it!” Mallinson threw down his cigarette and sprang', 'towards the door with eyes blazing. “You’re off your head!” he cried', 'wildly. “You’re mad, Conway, that’s what’s the matter with you! I know', 'you’re always calm, and I’m always excited, but I’m sane, at any rate,',

In [7]:
from bs4 import BeautifulSoup
import re

class Section:
    def __init__(self, title, body):
        self.title = title
        self.body = body

class HTMLSectionProcessor:
    def __init__(self, html, tags):
        self.html = html
        self.tags = tags

    def _find_section_boundaries(self):
        tags_regex = '|'.join(self.tags)
        pattern = re.compile(f"<(/)?({tags_regex})(:\\d*)?>")
        return [(bool(match.group(1)), match.group(2), match.span()) for match in pattern.finditer(self.html)]


    # _construct_sectionsメソッドの修正例
    def _construct_sections(self, boundaries):
        sections = []
        open_tag = None
        section_start = None
        for is_closing, tag_name, span in boundaries:
            if not is_closing and open_tag is None:
                open_tag = tag_name
                section_start = span[1]  # タグの終了直後からセクションを開始
            elif is_closing and open_tag == tag_name:
                sections.append((tag_name, self.html[section_start:span[0]].strip()))
                open_tag = None
        return sections
    
    # extract_content_from_sectionメソッドの調整
    @staticmethod
    def extract_content_from_section(section):
        soup = BeautifulSoup(section, 'html.parser')
        title, body = extract_title_or_body(soup, 'TITLE', True), extract_title_or_body(soup, 'body')
        
        if not title and not body:
            title = soup.get_text(strip=True)
        elif title and not body:
            body = [soup.get_text(strip=True)]
        
        return title, body

    @staticmethod
    def _extract_title_or_body(soup, tag, is_title=False):
        tag_func = lambda tag: tag.name.upper().endswith('TITLE') if is_title else tag.name.lower() == tag
        found_tag = soup.find(tag_func)
        if found_tag:
            text = found_tag.get_text(strip=True)
            return text if is_title else text.split("\n")
        return "" if is_title else []

    def process(self):
        boundaries = self._find_section_boundaries()
        raw_sections = self._construct_sections(boundaries)
        return [Section(*self.extract_content_from_section(section)) for _, section in raw_sections]

import re

def replace_single_newline_with_space(text):
    # 単独の改行を空白に置換。(?<!\n)と(?!\n)は直前直後に改行がないことを確認する
    return re.sub(r'(?<!\n)\n(?!\n)', ' ', text)



tag_list = ['PROLOGUE', 'CHAPTER', 'EPILOGUE', 'THE_END', 'TRANSCRIBER_NOTES']
processor = HTMLSectionProcessor(result, tag_list)
processed_sections = processor.process()
chapters = []
for section in processed_sections:
    print(f"Title: {section.title}")
    print("Body:", section.body)
    print("---")
    lines = []
    joined = "\n".join(section.body).split("\n\n")
    for one_line_text in joined: 
        one_line_text = replace_single_newline_with_space(one_line_text)
        lines.append(Line(one_line_text))
    chapters.append(Chapter(_title=section.title, contents=lines))

Title: PROLOGUE
Body: ['Cigars had burned low, and we were beginning to sample the disillusionment...']
---
Title: CHAPTER 1
Body: ['During that third week of May the situation in Baskul had become much worse...']
---
Title: CHAPTER 2
Body: ['The journey was expected to be long and perilous...', '', 'gave him a most charming smile, but her eyes were all for the boy.', '“What life?”', '', '“The life you’re thinking of . . . dinners . . . dances . . . polo . . .', 'all that. . . .”', '', '“But I never said anything about dances and polo! Anyhow, what’s wrong', 'with them? D’you mean that you’re not coming with me? You’re going to', 'stay here like the other two? Then at least you shan’t stop _me_ from', 'clearing out of it!” Mallinson threw down his cigarette and sprang', 'towards the door with eyes blazing. “You’re off your head!” he cried', 'wildly. “You’re mad, Conway, that’s what’s the matter with you! I know', 'you’re always calm, and I’m always excited, but I’m sane, at any rate,',

In [8]:
a = Book(_title = "lost horizon", contents = chapters)

In [9]:
a.word_count

567

In [10]:
a.show_conets

[['Cigars had burned low, and we were beginning to sample the disillusionment...'],
 ['During that third week of May the situation in Baskul had become much worse...'],
 ['The journey was expected to be long and perilous...',
  'gave him a most charming smile, but her eyes were all for the boy. “What life?”',
  '“The life you’re thinking of . . . dinners . . . dances . . . polo . . . all that. . . .”',
  '“But I never said anything about dances and polo! Anyhow, what’s wrong with them? D’you mean that you’re not coming with me? You’re going to stay here like the other two? Then at least you shan’t stop _me_ from clearing out of it!” Mallinson threw down his cigarette and sprang towards the door with eyes blazing. “You’re off your head!” he cried wildly. “You’re mad, Conway, that’s what’s the matter with you! I know you’re always calm, and I’m always excited, but I’m sane, at any rate, and you’re not! They warned me about it before I joined you at Baskul, and I thought they were wrong, 

In [12]:
aa = a.show_conets_with_title

In [13]:
import pprint

In [15]:
aa

'-----chapter : PROLOGUE-----\n\nCigars had burned low, and we were beginning to sample the disillusionment...\n\n-----chapter : CHAPTER 1-----\n\nDuring that third week of May the situation in Baskul had become much worse...\n\n-----chapter : CHAPTER 2-----\n\nThe journey was expected to be long and perilous...\ngave him a most charming smile, but her eyes were all for the boy. “What life?”\n“The life you’re thinking of . . . dinners . . . dances . . . polo . . . all that. . . .”\n“But I never said anything about dances and polo! Anyhow, what’s wrong with them? D’you mean that you’re not coming with me? You’re going to stay here like the other two? Then at least you shan’t stop _me_ from clearing out of it!” Mallinson threw down his cigarette and sprang towards the door with eyes blazing. “You’re off your head!” he cried wildly. “You’re mad, Conway, that’s what’s the matter with you! I know you’re always calm, and I’m always excited, but I’m sane, at any rate, and you’re not! They war

In [16]:
print(aa)

-----chapter : PROLOGUE-----

Cigars had burned low, and we were beginning to sample the disillusionment...

-----chapter : CHAPTER 1-----

During that third week of May the situation in Baskul had become much worse...

-----chapter : CHAPTER 2-----

The journey was expected to be long and perilous...
gave him a most charming smile, but her eyes were all for the boy. “What life?”
“The life you’re thinking of . . . dinners . . . dances . . . polo . . . all that. . . .”
“But I never said anything about dances and polo! Anyhow, what’s wrong with them? D’you mean that you’re not coming with me? You’re going to stay here like the other two? Then at least you shan’t stop _me_ from clearing out of it!” Mallinson threw down his cigarette and sprang towards the door with eyes blazing. “You’re off your head!” he cried wildly. “You’re mad, Conway, that’s what’s the matter with you! I know you’re always calm, and I’m always excited, but I’m sane, at any rate, and you’re not! They warned me about i